# Week 7 - Linear Programming

## Question 15.2

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional
requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem
with real data. The data is given in the file diet.xls.

1. Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP. Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)

2. Please add to your model the following constraints (which might require adding more variables) and solve the new model:

    a. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
    
    b. Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.
    
    c. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered  meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!]

In [39]:
import pandas as pd
import pulp

food = pd.DataFrame.from_csv('foods.csv', index_col=['food'])
food

,price_per_serving,calories,cholesterol,total_fat,sodium,carbohydrates,fiber,protien,vit_a,vit_c,calcium,iron
food,,,,,,,,,,,,
apple,0.24,81.4,0.0,0.5,0.0,21.0,3.7,0.3,73.1,7.9,9.7,0.2
apple_pie,0.16,67.2,0.0,3.1,75.4,9.6,0.5,0.5,35.2,0.9,3.1,0.1
bagel,0.16,78.0,0.0,0.5,151.4,15.1,0.6,3.0,0.0,0.0,21.0,1.0
baked_potato,0.06,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
banana,0.15,104.9,0.0,0.5,1.1,26.7,2.7,1.2,92.3,10.4,6.8,0.4
bean_and_bacon_soup,0.67,172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
beef_frankfurter,0.27,141.8,27.4,12.8,461.7,0.8,0.0,5.4,0.0,10.8,9.0,0.6
brocolli_frozen,0.16,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
butter,0.05,35.8,10.9,4.1,41.3,0.0,0.0,0.0,152.9,0.0,1.2,0.0


In [40]:
macro_limits = pd.DataFrame.from_csv('dietary_constraints.csv', index_col=['dietary_constraint'])
macro_limits

,calories,cholesterol,total_fat,sodium,carbohydrates,fiber,protien,vit_a,vit_c,calcium,iron
dietary_constraint,,,,,,,,,,,
min,1500,30,20,800,130,125,60,1000,400,700,10
max,2500,240,70,2000,450,250,100,10000,5000,1500,40


Now that we have the data imported, we can start setting our decision variables

In [41]:
# The quanity of each food
food_q = pulp.LpVariable.dicts("food_q", (food for food in food.index), lowBound=0, cat='Continuous')

# Whether or not the food is included in the diet
food_i = pulp.LpVariable.dicts("food_i", (food for food in food.index), cat='Binary')

In [46]:
# Obejective
# minimizie food_cost * food_quantity * food_included

# Instantiate our problem class
model = pulp.LpProblem("Army food rationaing minimization", pulp.LpMinimize)

# Objective Function
model += (
    pulp.lpSum([
        food.loc[f].price_per_serving * food_q[f]
        for f in food.index])
)



RecursionError: maximum recursion depth exceeded in comparison

In [43]:
# [food[f, 0] for f in food.index]
# food_q['apple']
food.loc['apple'].price_per_serving

0.23999999999999999